In [10]:
import pandas as pd
import os
import re

In [36]:
path = '/Users/joel/Desktop/SUTD /T5/SDS/MY OWN DATA/combined products done.xlsx'
# files =[]
# for file in os.listdir(path):
#     if file.endswith('csv'):
#         file_path = os.path.join(path, file)
#         df = pd.read_csv(file_path) 
#         file2 = file.split('.')[0]
#         product_name =re.split(r'\d', file2, 1)[0].strip() 
#         files.append(product_name)
data = pd.read_excel(path)
print(data.head())

         RecordedDate     Products Categories Feedback Categories  \
0 2024-04-05 09:21:00  Investments    Vickers         Application   
1 2024-04-05 09:23:00  Investments    Vickers     Process Related   
2 2024-04-05 09:24:00  Investments     Others              Others   
3 2024-04-05 09:26:00  Investments     Others              Others   
4 2024-04-05 09:27:00  Investments    Vickers         Application   

                                                 Q2b  Q6a  Q6b  Q10  Q19  
0         The app is easy to use and straightforward  NaN  NaN  NaN  NaN  
1                                   All info linked.  NaN  NaN  NaN  NaN  
2                                                NIL  NaN  NaN  NaN  NaN  
3                                                 na  NaN  NaN  NaN  NaN  
4  I was completely new to this but application w...  NaN  NaN  NaN  NaN  


In [38]:

data['combined_text'] = '[' + data['Products'].astype(str) + ' / ' + data['Categories'].astype(str) + ' / ' + data['Feedback Categories'].astype(str) + '] ' + data['Q2b'].astype(str)

print(data[['combined_text']].head())


                                       combined_text
0  [Investments / Vickers / Application] The app ...
1  [Investments / Vickers / Process Related] All ...
2                [Investments / Others / Others] NIL
3                 [Investments / Others / Others] na
4  [Investments / Vickers / Application] I was co...


In [39]:

for i in range(2,100):
    print(data.loc[i, 'combined_text'])


[Investments / Others / Others] NIL
[Investments / Others / Others] na
[Investments / Vickers / Application] I was completely new to this but application was easy to follow and quick to complete 
[Investments / Vickers / Process Related] it didnt take forever to open an account
[Investments / Vickers / Process Related] Everything could be done from my phone and the account was opened within a week.
[Investments / Vickers / Process Related] Very fast and easy
[Investments / Vickers / Process Related] Set up is very easy
[Investments / Vickers / Process Related] Every thing is so simple and easy to understand
[Investments / Vickers / Process Related] I feel very easy to check my application status.
[Investments / Vickers / Process Related] It's relatively straightforward and simple 
[Investments / Vickers / Process Related] Information is patchy and it’s all over the place. I have to Google for other sources to get the procedures
[Investments / Vickers / Process Related] It is fast and s

In [40]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    return text

data['cleaned_text'] = data['combined_text'].apply(clean_text)









In [43]:
from sklearn.model_selection import train_test_split

data = data.dropna(subset=['Products', 'Q2b'])

X = data[['Q2b', 'Categories', 'Feedback Categories']]
y = data['Products']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

preprocessor = ColumnTransformer(
    transformers=[
        ('txt', TfidfVectorizer(ngram_range=(1, 3)), 'Q2b'),
        ('cat', OneHotEncoder(), ['Categories', 'Feedback Categories'])
    ])

clf = make_pipeline(preprocessor, RandomForestClassifier())
clf.fit(X_train, y_train)
print("Model accuracy on test set:", clf.score(X_test, y_test))





Model accuracy on test set: 0.7461538461538462


In [47]:
print(len(X_test))
print(len(y))

130
517


In [44]:

y_pred = clf.predict(X_test)
print(y_pred)


['DBS Treasures' 'DBS Treasures' 'DBS Treasures' 'DBS Treasures'
 'DBS Treasures' 'DBS Treasures' 'DBS Treasures' 'DBS Treasures'
 'DBS Treasures' 'DBS Treasures' 'DBS Treasures' 'DBS Treasures'
 'DBS Treasures' 'DBS Treasures' 'Investments' 'DBS Treasures'
 'DBS Treasures' 'Investments' 'DBS Treasures' 'DBS Treasures'
 'DBS Treasures' 'DBS Treasures' 'DBS Treasures' 'DBS Treasures'
 'DBS Treasures' 'DBS Treasures' 'DBS Treasures' 'DBS Treasures'
 'DBS Treasures' 'DBS Treasures' 'DBS Treasures' 'DBS Treasures'
 'DBS Treasures' 'Investments' 'DBS Treasures' 'DBS Treasures'
 'DBS Treasures' 'DBS Treasures' 'DBS Treasures' 'DBS Treasures'
 'DBS Treasures' 'DBS Treasures' 'DBS Treasures' 'DBS Treasures'
 'DBS Treasures' 'DBS Treasures' 'DBS Treasures' 'DBS Treasures'
 'DBS Treasures' 'DBS Treasures' 'DBS Treasures' 'DBS Treasures'
 'DBS Treasures' 'DBS Treasures' 'DBS Treasures' 'DBS Treasures'
 'DBS Treasures' 'DBS Treasures' 'DBS Treasures' 'DBS Treasures'
 'DBS Treasures' 'DBS Treasures

In [50]:
import pandas as pd
results = X_test.copy()
results['Actual'] = y_test
results['Predicted'] = y_pred
results.reset_index(drop=True, inplace=True)
print(results[['Q2b', 'Actual', 'Predicted']])
print(len(results['Predicted']))



                                                   Q2b         Actual  \
0    Prompt in replying my query and efficient when...  DBS Treasures   
1    Darren started the Financial Planning exercise...  DBS Treasures   
2                                    Very good returns  DBS Treasures   
3    1. Recommend the type of products base on my n...   DBS Treasure   
4                Patient and attention to my request.   DBS Treasures   
..                                                 ...            ...   
125                                    Singpass linked    Investments   
126                  Patience and able to give advice   DBS Treasures   
127  She attentively checks my current financial st...  DBS Treasures   
128  RM brought in an insurance specialist who trie...   DBS Treasure   
129                                        No comment   DBS Treasures   

         Predicted  
0    DBS Treasures  
1    DBS Treasures  
2    DBS Treasures  
3    DBS Treasures  
4    DBS Treasures

In [52]:

results.to_csv('/Users/joel/Desktop/SUTD /T5/SDS/MY OWN DATA/Feedback_Predictions.csv', index=False)
